In [ ]:
import numpy as np
from scipy.signal import welch
from muselsl import stream, list_muses, view

In [ ]:
from pylsl import StreamInlet, resolve_byprop
from scipy.signal import welch, butter, lfilter

We will be using the butterwoth filter

In [2]:
import numpy as np
from pylsl import StreamInlet, resolve_byprop
from scipy.signal import welch, butter, lfilter
import pyautogui


# Constants
FS = 256  # Hz, sampling frequency
BETA_LOW, BETA_HIGH = 13, 30  # Beta band range in Hz
BUFFER_LENGTH = 5 * FS  # 5 seconds of data
HIGH_CONCENTRATION_THRESHOLD = 0.01  # Example threshold, calibrate this

# Design the Butterworth bandpass filter
def butter_bandpass(lowcut, highcut, fs, order=5):
    nyq = 0.5 * fs
    low = lowcut / nyq
    high = highcut / nyq
    b, a = butter(order, [low, high], btype='band')
    return b, a

# Apply the bandpass filter
def bandpass_filter(data, lowcut, highcut, fs, order=5):
    b, a = butter_bandpass(lowcut, highcut, fs, order=order)
    y = lfilter(b, a, data, axis=0)
    return y

# Calculate the beta band power
def beta_band_power(data, fs):
    freqs, psd = welch(data, fs=fs, axis=0)
    idx_beta = np.logical_and(freqs >= BETA_LOW, freqs <= BETA_HIGH)
    beta_power = np.mean(psd[idx_beta], axis=0)
    return beta_power

# Main processing function
def process_eeg_data(inlet):
    # Initialize buffer
    eeg_buffer = np.zeros((BUFFER_LENGTH, inlet.info().channel_count()))
    
    while True:
        # Pull sample from inlet
        sample, timestamp = inlet.pull_sample(timeout=1.0)
        if sample:
            # Update the buffer
            eeg_buffer = np.roll(eeg_buffer, -1, axis=0)
            eeg_buffer[-1, :] = sample

            # Check if buffer is full
            if np.all(eeg_buffer[:, 0] != 0):
                # Preprocess the buffer - apply bandpass filter
                filtered_data = bandpass_filter(eeg_buffer, BETA_LOW, BETA_HIGH, FS)

                # Calculate beta band power
                beta_power = beta_band_power(filtered_data, FS)

                # Determine high concentration
                high_concentration = beta_power > HIGH_CONCENTRATION_THRESHOLD

                # Output high concentration status
                print(f"High concentration detected: {high_concentration}")
                # Here you would trigger a game action if high concentration is detected
                # trigger_game_action('move') if high_concentration else None
                # Give yourself a couple of seconds to switch to the game window
                print("Switch to the game window. The spacebar will be pressed automatically ...")
                #sleep(5)
                pyautogui.press('space')

# Resolve the stream
print("Looking for an EEG stream...")
streams = resolve_byprop('type', 'EEG', timeout=2)

if streams:
    inlet = StreamInlet(streams[0])
    print("Stream found and connected")
    process_eeg_data(inlet)  # Start processing EEG data
else:
    print("No streams found")

Looking for an EEG stream...
Stream found and connected
High concentration detected: [ True  True  True  True  True]
Switch to the game window. The spacebar will be pressed automatically ...
High concentration detected: [ True  True  True  True  True]
Switch to the game window. The spacebar will be pressed automatically ...
High concentration detected: [ True  True  True  True  True]
Switch to the game window. The spacebar will be pressed automatically ...
High concentration detected: [ True  True  True  True  True]
Switch to the game window. The spacebar will be pressed automatically ...
High concentration detected: [ True  True  True  True  True]
Switch to the game window. The spacebar will be pressed automatically ...
High concentration detected: [ True  True  True  True  True]
Switch to the game window. The spacebar will be pressed automatically ...
High concentration detected: [ True  True  True  True  True]
Switch to the game window. The spacebar will be pressed automatically ...

KeyboardInterrupt: 

In [1]:
import pyautogui
from time import sleep

# Give yourself a couple of seconds to switch to the game window
print("Switch to the game window. The spacebar will be pressed automatically after 5 seconds...")
sleep(5)

# Press the spacebar - this should make the character in the game jump
pyautogui.press('space')
print("The spacebar was pressed.")

# Run this block to see if the Dino jumps. Make sure the Dino game window is active and in focus.


Switch to the game window. The spacebar will be pressed automatically after 5 seconds...
The spacebar was pressed.
